In [74]:
! pip install -U imbalanced-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.0/258.0 kB 2.2 MB/s eta 0:00:00
  Attempting uninstall: imbalanced-learn
    Found existing installation: imbalanced-learn 0.10.1
    Uninstalling imbalanced-learn-0.10.1:
      Successfully uninstalled imbalanced-learn-0.10.1


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from imblearn.over_sampling import RandomOverSampler

In [36]:
df = pd.read_csv("/content/training.csv", index_col=0)

In [37]:
df.head()

,1,2,3,4,5,6,7,8,9,10,...,292,293,294,295,296,297,298,299,300,category
0,-0.075978,0.042700,-0.044242,0.031259,0.009310,-0.028852,0.030226,0.077606,0.005035,-0.819824,...,-0.042395,0.007928,-0.020569,0.116584,0.176855,-0.002770,-0.133249,0.019346,-0.003295,acq
1,-0.022188,0.050306,-0.072518,0.032942,-0.066491,-0.023295,0.016003,0.005223,-0.051511,-0.884397,...,-0.073233,-0.016061,0.046810,0.165003,0.279755,-0.102694,-0.136777,0.000949,0.042158,acq
2,-0.012057,0.023780,0.017903,0.042739,-0.009014,-0.114575,-0.014289,-0.004379,-0.077230,-0.862342,...,-0.006361,0.070082,0.034964,0.093347,0.257715,0.061832,-0.163226,-0.179587,0.019424,acq
3,-0.066585,0.001571,-0.058811,0.019376,0.030834,-0.024728,0.039057,0.041611,0.027092,-0.816220,...,-0.045804,0.031745,0.079883,0.122354,0.108574,-0.013479,-0.119203,-0.060059,0.045564,acq
4,0.142390,-0.006126,0.006468,0.037687,-0.016991,-0.089097,0.034997,0.068066,0.035505,-0.512829,...,0.039550,-0.024561,0.062537,0.033188,0.201466,-0.022500,-0.056796,-0.047506,0.030917,acq


In [38]:
df.columns

Index(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10',
       ...
       '292', '293', '294', '295', '296', '297', '298', '299', '300',
       'category'],
      dtype='object', length=301)

In [39]:
X_train = df.drop(["category"], axis=1)
Y_train = df[["category"]]

In [40]:
testing_df = pd.read_csv("/content/testing.csv", index_col=0)

X_valid, X_test, Y_valid, Y_test = train_test_split(testing_df.drop(["category"], axis=1), testing_df[["category"]], test_size=0.7, random_state=0)

In [41]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(Y_train)
Y_train = encoded_labels
Y_valid = label_encoder.transform(Y_valid)
Y_test = label_encoder.transform(Y_test)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


In [42]:
Y_train

array([ 0,  0,  0, ..., 90, 90, 90])

In [43]:
# # Assuming X_train and X_test contain your input data
# X_train = np.array(X_train)  # Convert to numpy array if not already
# X_test = np.array(X_test)

# # Reshape the input data to have the shape (batch_size, 300)
# X_train = X_train.reshape(-1)
# X_test = X_test.reshape(-1)

In [67]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(300, activation='relu', input_shape=(300,)),  # Input layer with 300 neurons
    tf.keras.layers.Dense(300, activation='relu'),  # Hidden layer with 300 neurons
    tf.keras.layers.Dense(300, activation='relu'),  # Hidden layer with 300 neurons
    tf.keras.layers.Dense(300, activation='relu'),  # Hidden layer with 300 neurons
    tf.keras.layers.Dense(91, activation='softmax')
])


model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=[tf.keras.metrics.F1Score(average='macro', name='f1_score', dtype=None)]
)

In [45]:
from tensorflow.keras.utils import to_categorical
Y_train = to_categorical(Y_train, num_classes=91)
Y_valid = to_categorical(Y_valid, num_classes=91)
Y_test = to_categorical(Y_test, num_classes=91)

In [68]:
to_categorical(Y_train[1], num_classes=91).argmax()

1

In [69]:
label_encoder.inverse_transform([0])

array(['acq'], dtype=object)

In [70]:
model.evaluate(X_train, Y_train.astype('float32'))

357/357 [==============================] - 2s 4ms/step - loss: 4.5101 - f1_score: 0.0020


[4.510107040405273, 0.00195601349696517]

In [71]:
model.evaluate(X_valid, Y_valid)

38/38 [==============================] - 0s 3ms/step - loss: 4.5124 - f1_score: 7.8882e-04


[4.5124335289001465, 0.0007888195104897022]

In [72]:
model.fit(X_train, Y_train.astype('float32'), batch_size=500, epochs=1000, validation_data=(X_valid, Y_valid))

Epoch 1/1000
23/23 [==============================] - 2s 36ms/step - loss: 3.0760 - f1_score: 0.0124 - val_loss: 2.3556 - val_f1_score: 0.0176
Epoch 2/1000
23/23 [==============================] - 1s 32ms/step - loss: 2.0191 - f1_score: 0.0282 - val_loss: 1.8797 - val_f1_score: 0.0394
Epoch 3/1000
23/23 [==============================] - 1s 30ms/step - loss: 1.5438 - f1_score: 0.0526 - val_loss: 1.5490 - val_f1_score: 0.0594
Epoch 4/1000
23/23 [==============================] - 1s 31ms/step - loss: 1.2871 - f1_score: 0.0772 - val_loss: 1.4118 - val_f1_score: 0.0741
Epoch 5/1000
23/23 [==============================] - 1s 31ms/step - loss: 1.1609 - f1_score: 0.0952 - val_loss: 1.2843 - val_f1_score: 0.0880
Epoch 6/1000
23/23 [==============================] - 1s 30ms/step - loss: 1.0651 - f1_score: 0.1129 - val_loss: 1.2550 - val_f1_score: 0.1027
Epoch 7/1000
23/23 [==============================] - 1s 32ms/step - loss: 1.0161 - f1_score: 0.1284 - val_loss: 1.2525 - val_f1_score: 0.1050

In [73]:
model.evaluate(X_test, Y_test)

89/89 [==============================] - 0s 3ms/step - loss: 2.7114 - f1_score: 0.2817


[2.711427927017212, 0.2817162871360779]

In [64]:
metric = tf.metrics.F1Score()
Y_pred = model.predict(X_test)
metric.update_state(Y_test, Y_pred)

89/89 [==============================] - 0s 4ms/step


In [65]:
predictions = model.predict(X_test)
encoded_classes = tf.argmax(predictions, axis=1).numpy()
label_encoder.inverse_transform(encoded_classes)

89/89 [==============================] - 0s 3ms/step


array(['earn', 'money-fx', 'cpi', ..., 'earn', 'crude', 'wheat'],
      dtype=object)

In [66]:
model.save('/content/fourth_model.keras')

In [55]:

def predict_classes(df):
    predictions = model.predict(X_test)
    encoded_classes = tf.argmax(predictions, axis=1).numpy()

    return label_encoder.inverse_transform(encoded_classes)


In [56]:
predict_classes(Y_test)

89/89 [==============================] - 0s 3ms/step


array(['earn', 'money-fx', 'grain', ..., 'earn', 'crude', 'wheat'],
      dtype=object)